Task: Data extraction through API (data source: Electrica De Espana)

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
def demand_evolution(start_date, end_date):
    #Enter the start_date and end_date in the following format: YYYY-MM-DDTHH:MM
    
    response = requests.get(f"https://apidatos.ree.es/en/datos/demanda/evolucion-renovable-no-renovable?start_date={start_date}&end_date={end_date}&time_trunc=day")
    response_json=response.json()
    data=response_json["included"][0]["attributes"]["values"]
    #Formating
    evolution=pd.DataFrame(data).drop("percentage", axis=1)
    evolution["value"]=evolution["value"].round(2)
    evolution['datetime'] = pd.to_datetime(evolution["datetime"],format='%Y-%m')
    evolution=evolution.rename(columns={'value': 'Value', "datetime":"Date"})
    return (evolution)

In [ ]:
# The very first object which we get when accessing the json file: nested dictionarties

response = requests.get("https://apidatos.ree.es/en/datos/generacion/evolucion-renovable-no-renovable?start_date=2020-01-01T00:00&end_date=2020-06-15T00:00&time_trunc=day")
response_json=response.json()
response_json

In [14]:
#levels down to the attributes item:

data=response_json["included"][0]["attributes"]
data

{'title': 'Renewable',
 'description': 'Renewable energy: includes hydro, hydro, wind, solar photovoltaic, solar thermal, other renewables and renewable waste.',
 'color': '#92d050',
 'type': 'Renovable',
 'magnitude': None,
 'composite': False,
 'last-update': '2020-06-15T07:52:27.000+02:00',
 'values': [{'value': 213766.0045,
   'percentage': 0.35901114116181204,
   'datetime': '2020-01-01T00:00:00.000+01:00'},
  {'value': 269999.7285,
   'percentage': 0.3814219026715909,
   'datetime': '2020-01-02T00:00:00.000+01:00'},
  {'value': 265464.275,
   'percentage': 0.3870897904935839,
   'datetime': '2020-01-03T00:00:00.000+01:00'},
  {'value': 301826.6765,
   'percentage': 0.44873595615348133,
   'datetime': '2020-01-04T00:00:00.000+01:00'},
  {'value': 232994.6835,
   'percentage': 0.3809623883144443,
   'datetime': '2020-01-05T00:00:00.000+01:00'},
  {'value': 224409.3195,
   'percentage': 0.3650340883841512,
   'datetime': '2020-01-06T00:00:00.000+01:00'},
  {'value': 247350.2735,
   

In [15]:
# From the below dataframe we only need the title column, and to unstack the values (list of dictionaries) for each day 
RNR_evolution=pd.DataFrame(data)
RNR_evolution

,title,description,color,type,magnitude,composite,last-update,values
0,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 213766.0045, 'percentage': 0.3590111..."
1,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 269999.7285, 'percentage': 0.3814219..."
2,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 265464.275, 'percentage': 0.38708979..."
3,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 301826.6765, 'percentage': 0.4487359..."
4,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 232994.6835, 'percentage': 0.3809623..."
...,...,...,...,...,...,...,...,...
162,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 357927.856, 'percentage': 0.53236108..."
163,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 402942.102, 'percentage': 0.58246824..."
164,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 336162.056, 'percentage': 0.53315539..."
165,Renewable,"Renewable energy: includes hydro, hydro, wind,...",#92d050,Renovable,None,False,2020-06-15T07:52:27.000+02:00,"{'value': 246801.641, 'percentage': 0.43984910..."


In [16]:
rnb=response_json["included"][0]["attributes"]["title"] # We need to add this to each renewable value
nrnb=response_json["included"][1]["attributes"]["title"]# We need to add this to each non-renewable value 
rnb_values=response_json["included"][0]["attributes"]["values"] #list of dictionaries for renewable evolution
nrnb_values=response_json["included"][1]["attributes"]["values"] # list of dictionaries for non-renewable evolution

rnb_values

[{'value': 213766.0045,
  'percentage': 0.35901114116181204,
  'datetime': '2020-01-01T00:00:00.000+01:00'},
 {'value': 269999.7285,
  'percentage': 0.3814219026715909,
  'datetime': '2020-01-02T00:00:00.000+01:00'},
 {'value': 265464.275,
  'percentage': 0.3870897904935839,
  'datetime': '2020-01-03T00:00:00.000+01:00'},
 {'value': 301826.6765,
  'percentage': 0.44873595615348133,
  'datetime': '2020-01-04T00:00:00.000+01:00'},
 {'value': 232994.6835,
  'percentage': 0.3809623883144443,
  'datetime': '2020-01-05T00:00:00.000+01:00'},
 {'value': 224409.3195,
  'percentage': 0.3650340883841512,
  'datetime': '2020-01-06T00:00:00.000+01:00'},
 {'value': 247350.2735,
  'percentage': 0.3353956991167547,
  'datetime': '2020-01-07T00:00:00.000+01:00'},
 {'value': 253539.0765,
  'percentage': 0.3454292587402804,
  'datetime': '2020-01-08T00:00:00.000+01:00'},
 {'value': 306565.7375,
  'percentage': 0.41011977162160906,
  'datetime': '2020-01-09T00:00:00.000+01:00'},
 {'value': 348793.0085,
  

In [ ]:
#Ignore this cell for now!

data=response_json["included"][0]["attributes"]["values"]+response_json["included"][1]["attributes"]["values"]
data